In [ ]:
import cv2 as cv
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from ipywidgets import interact, fixed
import ipywidgets as widgets
import functions

%load_ext autoreload
%autoreload 2

In [ ]:
equalizeHist = True
crop = True
align= True
valid = cv.imread("template.png", 0)
defected = cv.imread("defected.png", 0)
if equalizeHist:
    valid = cv.equalizeHist(valid)
    defected = defected = cv.equalizeHist(defected)
if crop:
    valid = functions.cropper1(valid, y1_i=5, thresh_border=19815)
    defected = functions.cropper1(defected, y1_i=5, thresh_border=19815)
if align:
    defected = functions.aligner(valid, defected)[0] #aligner returns tuple  (img, aligning matrix)


fig, ax = plt.subplots(1,2, figsize = (15,15))
ax[0].imshow(valid, cmap='gray')
ax[1].imshow(defected, cmap='gray')
        


# DIFFERENCE

In [ ]:
@interact(er_shape=(0,2), er_size=(1,10), kernel_size=widgets.IntSlider(value=21, min=5, max=30,step=2), 
          closing_size=(1,10), opening_size=(1,10), simple_method_thresh=(10,250), threshold=1000)
def difference(
    er_shape=2,
    er_size=4,
    kernel_size=21,
    closing=True,
    closing_size=3,
    opening=True,
    opening_size=5,
    method="simple", # can be simple or complex
    simple_method_thresh=50, #threshold for binaryzation
    threshold=1000, # theshold by which we classify as defected
    plot=True
):
    """
    Added decision based on the sum, if sum of the subtracted images is above threshold we return Defected state!

    Im writes and reads are the most expensive in case of time needed to perform it
    """
    template = (
            cv.cvtColor(valid, cv.COLOR_BGR2GRAY)
            if len(valid.shape) == 3
            else valid
        )

    to_be_inspected = (
        cv.cvtColor(defected, cv.COLOR_BGR2GRAY)
        if len(defected.shape) == 3
        else defected
    ) 
        
    if method == "complex":
        template = cv.adaptiveThreshold(
            template,
            255,  # maxValue: Non-zero value assigned to the pixels for which the condition is satisfied
            cv.ADAPTIVE_THRESH_GAUSSIAN_C,  #  Adaptive thresholding algorithm to use
            cv.THRESH_BINARY,  # Thresholding type that must be either THRESH_BINARY or THRESH_BINARY_INV
            kernel_size,  # 	Size of a pixel neighborhood that is used to calculate a threshold value for the pixel: 3, 5, 7, and so on
            1,  # Constant subtracted from the mean or weighted mean
        )  # The bigger kernel the better binaryzation is from experiments
        to_be_inspected = cv.adaptiveThreshold(
            to_be_inspected,
            255,  # maxValue: Non-zero value assigned to the pixels for which the condition is satisfied
            cv.ADAPTIVE_THRESH_GAUSSIAN_C,  #  Adaptive thresholding algorithm to use
            cv.THRESH_BINARY,  # Thresholding type that must be either THRESH_BINARY or THRESH_BINARY_INV
            kernel_size,  # Size of a pixel neighborhood that is used to calculate a threshold value for the pixel: 3, 5, 7, and so on
            1,  # Constant subtracted from the mean or weighted mean
        )
        bitwiseXor = cv.bitwise_xor(template, to_be_inspected)

    elif method == "simple":
        bitwiseXor = cv.bitwise_xor(template, to_be_inspected)
        bitwiseXor[bitwiseXor >= simple_method_thresh] = 255
        bitwiseXor[bitwiseXor < simple_method_thresh] = 0

    # anchor: position within the element
    # The default value [−1,−1] means that the anchor is at the center
    element = cv.getStructuringElement(
        er_shape, ksize=(er_size, er_size), anchor=(-1, -1)
    )

    # We perform some morphological operations to clean our result_img from noise
    result_img = cv.erode(bitwiseXor, element)  # First perform erosion
    if opening:
        result_img = cv.morphologyEx(
            result_img,
            cv.MORPH_OPEN,
            np.ones((opening_size, opening_size), np.uint8),
        )
    if closing:
        result_img = cv.morphologyEx(
            result_img,
            cv.MORPH_CLOSE,
            np.ones((closing_size, closing_size), np.uint8),
        )

    sum_ = np.sum(result_img)
    if plot:
        fig,ax = plt.subplots(1,1, figsize=(10,10))
        plt.imshow(result_img, cmap='gray')
    # Based on sum of the pixels we decide defected or not if yes then we save inference to the file
    if sum_ > threshold:
        return "Defected", result_img
    else:
        return "Valid", result_img

## GENERATING BOXES

In [ ]:
@interact(er_shape=(0,2), er_size=(1,10), 
          closing_size=(1,10), opening_size=(1,10), simple_method_thresh=(10,250), shift=(0,20), low_area_thresh=(100,2000))
def generate_boxes(
    er_shape=2,
    er_size=3,
    closing=True,
    closing_size=10,
    opening=True,
    opening_size=2,
    method="simple",
    simple_method_thresh=50,
    shift=5,
    low_area_thresh=800,
):

    fig, ax = plt.subplots(1,1, figsize=(15,15))
    
    diff_img = difference(er_shape, er_size, plot=False, closing=closing, closing_size=closing_size,simple_method_thresh=simple_method_thresh, 
                          opening=opening, opening_size=opening_size, method=method)[1]  # index 0 is decision index 1 is image

    contours, hierarchy = cv.findContours(
        diff_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
    )  # It may be possible to use hierarchy later is we would like to place a few small objects that are close together inside one rectangle

    boxes = []
    
    for c in contours:
        x, y, w, h = cv.boundingRect(c)
        area = w * h
        if (
            area > low_area_thresh
        ):  # If there are some noisy very small contours we neglect them
            boxes.append((x -shift, y - shift, w+ 2 * shift, h+ 2* shift))
            # ! Experimenting with using shift parameter so that create a little bit bigger boxes
    diff_img = cv.cvtColor(diff_img, cv.COLOR_GRAY2RGB)
    for box in boxes:
        x,y,w,h = box
        cv.rectangle(diff_img, (x, y), (x + w, y + h), (0,255,0), 3)
    ax.imshow(diff_img)
    
    return boxes


## ROI CROPPER
### There is also cropper using color mask but for grayscale I think it is not useful

In [ ]:
@interact(x1_i=(0,50), x2_i=(-50,-1), y1_i=(0,50), y2_i=(-50,-1), canny_thresh_low=(0,255), canny_thresh_top=(0,255))
def cropper1(
    x1_i=0, # These are which edge from left/right/top/bottom to take
    x2_i=-1,
    y1_i=3,
    y2_i=-1,
    canny_thresh_low=208, # these two parameters determine thresholds for identifind edges
    canny_thresh_top=255,
    thresh_border=7500,
    crop_valid=True
):
    """
    For this to work a little bit better, after creating a mask we can try to do some Morphological operations
    """
    fig, ax = plt.subplots(1,3, figsize=(20,20))
    
    valid = cv.imread("template.png", 0)
    defected = cv.imread("defected.png", 0)
    if crop_valid:
        to_be_cropped = valid
    else:
        to_be_cropped = defected
        
    mask = cv.Canny(image=to_be_cropped, threshold1=canny_thresh_low, threshold2=canny_thresh_top)

    # cv.imwrite(f"outcome/mask1.png", mask)
    sumX = np.sum(
        mask, axis=0
    )  # collapsing rows, values of pixels from columns are summed
    indicesX = np.argwhere(sumX > thresh_border)
    x1 = indicesX[x1_i][
        0
    ]  # getting x1_i-th value from the left as a top left x coordinate
    x2 = indicesX[x2_i][0]

    sumY = np.sum(mask, axis=1)  # collapsing columns, rowss are summed
    indicesY = np.argwhere(sumY > thresh_border)  # the same story as above
    y1 = indicesY[y1_i][0]
    y2 = indicesY[y2_i][0]

    cropped = to_be_cropped[y1 : y2 + 1, x1 : x2 + 1]  # cropping the image
    
    ax[0].imshow(mask, cmap='gray')
    ax[1].imshow(to_be_cropped,cmap='gray')
    rect = patches.Rectangle((x1,y1), x2-x1, y2-y1, linewidth=2, facecolor='none', edgecolor='r')
    ax[1].add_patch(rect)
    ax[2].imshow(cropped,cmap='gray')
    # cv.imwrite(f"outcome/cropped1.png", cropped)
    #return cropped

## Template Matching

In [ ]:
#Difference between method 1 and method 3. Method 1 - the grater the score the more dissimilar objects. Method 3 - the greater the score the more similar objects are
@interact(matching_method= widgets.IntSlider(value=3, min=1, max=5,step=2), template_size=(50,250), min_difference=widgets.BoundedFloatText(min=0.0, max=1.0, step=0.01, value=0.95))
def template_matching(matching_method=3, template_size=150, min_difference=0.99, using_boxes=True):
    template = (
        cv.cvtColor(valid, cv.COLOR_BGR2GRAY)
        if len(valid.shape) == 3
        else valid
    )
    img_gray = cv.cvtColor(defected, cv.COLOR_BGR2GRAY) if len(defected.shape) == 3 else defected

    # font = cv.FONT_HERSHEY_SIMPLEX

    matching = []
    boxes = [(426, 221, 116, 76)] #Unfortunately we need to copy array generated from box_generator here
    print(boxes)
    boxes_to_plot = []
    defects = 0  # to count how many defect we have
    with open("template_matching.txt", "w") as f:
        if using_boxes:
            for box in boxes:
                x, y, w, h = box

                temp = template[
                    y : y + h, x : x + w
                ]  # first coordinate is y (height) axis second is x (width) axis
                deff = img_gray[y : y + h, x : x + w]  # crop image

                res_def = cv.matchTemplate(
                    deff, temp, matching_method
                )  # check how similar they're

                max_def = res_def[0][0]  # round it
                matching.append(max_def)

                if max_def < min_difference:
                    defects += 1
                    f.write(f"{x}, {y}, {x+w}, {y+h}, {max_def}\n")
                    boxes_to_plot.append([*box, max_def])
                    
        else:
            for y in range(
                0, template.shape[0], template_size
            ):  # get every possible square
                for x in range(0, template.shape[1], template_size):
                    temp = template[
                        y : y + template_size, x : x + template_size
                    ]  # first coordinate is y (height) axis second is x (width) axis
                    deff = img_gray[
                        y : y + template_size, x : x + template_size
                    ]  # crop image

                    res_def = cv.matchTemplate(
                        deff, temp, matching_method
                    )  # check how similar they're

                    max_def = res_def[0][0] # it return 2d array with 1 element
                    matching.append(max_def)

                    if max_def < min_difference:
                        defects += 1
                        f.write(
                            f"{x}, {y}, {x+template_size}, {y+template_size}, {max_def}\n"
                        )
                        boxes_to_plot.append([x,y,template_size,template_size, max_def])

        img_gray = cv.cvtColor(img_gray, cv.COLOR_GRAY2RGB)
        for box in boxes_to_plot:
            print(box)
            x, y, w, h, sim = box
            cv.rectangle(img_gray, (x, y), (x + w, y + h), 255, 3)
            cv.putText(
                img_gray,
                str(np.round(sim, 2)),  # Text string to be drawn.
                (x + 20, y + 30),  # Bottom-left corner.
                cv.FONT_HERSHEY_SIMPLEX,  # Font type, see HersheyFonts.
                1,  # Font scale factor that is multiplied by the font-specific base size.
                255,  # Text color.
                2,  # Thickness of the lines used to draw a text.
                cv.LINE_AA,  # Line type.
            )

    fig,ax = plt.subplots(1,1, figsize=(10,10))
    ax.imshow(img_gray)
    if defects > 0:
        return "Defected", matching
    else:
        return "Valid", matching